In [11]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, confusion_matrix, accuracy_score, recall_score
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder, OrdinalEncoder
import datetime
import os
import json
import csv

In [12]:
# Parámetros
inputFolder = "1-input"
processFolder = "2-process"
outputFolder = "3-output"
logsFolder = "4-logs"

dataVisualizationTopLimit = 20

testSize = 0.25
randomState = 0
partitionsNumber = 4

In [13]:
# Funciones utilitarias
def readData(filePath, delimiter, encoding, header):
  data = pd.read_csv(filePath, delimiter=delimiter,encoding=encoding, header=header)
  return data

def writeJson(data, pathJson, encoding='utf-8'):
  with open(pathJson, 'w', encoding=encoding) as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

def writeCsv(data, pathCsv, encoding='utf-8'):
  with open(pathCsv, 'w', newline='', encoding=encoding) as f:
    if data:
      writer = csv.DictWriter(f, fieldnames=data[0].keys(), lineterminator='\n')
      writer.writeheader()
      writer.writerows(data)
    else:
      f.write("")

In [14]:
def preprocessingData(df):
  # Aca no se realizará ni parseo ni nada, ya vendrá desde la fuente
  # Solo se leerá, se dividirá y se entrenará

  # Balanceo de datos: Sobremuestreo aleatorio (oversampling)
  objectiveColumn = "hired"
  class0, class1 = df[objectiveColumn].value_counts()
  dfClass0 = df[df[objectiveColumn] == 0]
  dfClass1 = df[df[objectiveColumn] == 1]
  dfClass1Sampled = dfClass1.sample(class0, replace=True)
  df = pd.concat([dfClass0, dfClass1Sampled],axis=0)

  # Desordenando la data para evitar sesgos(filas)
  df = df.sample(frac = 1, random_state=randomState).reset_index(drop=True)  

  # Aplicando OrdinalEncoding a las variables categóricas ordinales()
  categoricalColumns = [columnName for columnName, columnType in df.dtypes.to_dict().items() if columnName not in [ "hired" ] and columnType == "object" ]
  categoricalOrdinalColumns = [columnName for columnName in categoricalColumns if columnName in [ "lastEducationStatus", "lastEducationDegree" ]]

  # OrdinalEncoder para la columna lastEducationStatus
  encoder = OrdinalEncoder(categories=[[ "Abandonado", "En Curso", "Graduado" ]])
  encoder.fit(df[["lastEducationStatus"]])
  df["lastEducationStatus"] = encoder.transform(df[["lastEducationStatus"]])

  # OrdinalEncoder para la columna lastEducationDegree
  encoder = OrdinalEncoder(categories=[[ "Otro", "Secundario", "Terciario/Tecnico", "Universitario", "Posgrado", "Master", "Doctorado" ]])
  encoder.fit(df[["lastEducationDegree"]])
  df["lastEducationDegree"] = encoder.transform(df[["lastEducationDegree"]])

  #display(df)

  # Aplicando OneHotEncoding a las variables categóricas cardinales (transformación a numéricas mediante columnas)
  categoricalCardinalColumns = [columnName for columnName in categoricalColumns if columnName not in [ "lastEducationStatus", "lastEducationDegree" ]]
  for column in categoricalCardinalColumns:
    dummies = pd.get_dummies(df[[column]], prefix=column, dummy_na=True)
    df = pd.concat([df, dummies], axis = 1)
    df = df.drop(columns=[column])

  #display(df.dtypes)

  # Aplicando MinMaxScaler a las variables numéricas (normalización) (esto tambien incluye a lastEducationStatus y lastEducationDegree, ya numéricas)
  # Algunas quedaran en 0.9999, esto porque no todas manejan la misma escala (sin decimales, o solo un decimal)
  numericalColumns = [columnName for columnName, columnType in df.dtypes.to_dict().items() if columnName not in [ "hired" ] and columnType == "float64" ]
  #print(numericalColumns)
  for column in numericalColumns:
    df[column] = df[column].fillna(0.0)

  mms = MinMaxScaler()
  df[numericalColumns] = mms.fit_transform(df[numericalColumns])

  # Leyendo el numero de atributos
  """display(df)
  print(len(df))
  print(len(df.columns))"""
  
  # 19146 filas, 12403 columnas con el drop_first=True
  # 19146 filas, 12415 columnas sin el drop_first=True

  # Revision maximos y minimos (todos si estan entre 0 y 1)
  """df.loc['max'] = df.max()
  df.loc['min']= df.min()
  maxValue, minValue = max(list(df.iloc[len(df)-2])), min(list(df.iloc[len(df)-1]))
  print(maxValue, minValue)
  df = df.drop(['max', 'min'], axis=0)"""

  # Eliminando columnas con varianza cercana a cero (variables no afectan en el resultado del modelo)
  percentileNumber = 95
  df.loc['std'] = df.std()
  stdArray = df.iloc[len(df)-1]
  ninetyNinthPercentile = np.percentile(stdArray, percentileNumber)
  df = df.transpose()
  df = df[df["std"]>ninetyNinthPercentile]
  df = df.transpose()
  df = df.drop(['std'], axis=0)

  """display(df)
  print(len(df))
  print(len(df.columns))"""

  # Lectura de las variables de características y objetivo
  objectiveColumn = "hired"
  X = df.drop([objectiveColumn], axis=1)
  y = df[objectiveColumn]

  return X, y

In [15]:
def splitData(X, y, partitionNumber):
  # Dividiendo los dataframes de entrenamiento y prueba
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = testSize, random_state=randomState)

  if partitionNumber != 0:
    # Obteniendo el total de filas del dataframe de testeo
    X_train, X_test, y_train, y_test = X_train.reset_index(drop=True), X_test.reset_index(drop=True), y_train.reset_index(drop=True), y_test.reset_index(drop=True)
    totalRows = len(X_train)

    # Determinando el límite inferior (primera fila) de la partición
    bottomLimit = int(totalRows*testSize*(partitionNumber-1)) + 1

    # Determinando el límite superior (última fila) de la partición
    topLimit = int(totalRows*testSize*partitionNumber)

    # Determinando una lista con todas los numeros de las filas con la partición
    testList = [x for x in range(bottomLimit-1, topLimit)]

    # Determinando los dataframes de las categorías
    X_train, X_test = X_train[~X_train.index.isin(testList)], X_train[X_train.index.isin(testList)]

    # Determinando los dataframes de los objetivos
    y_train, y_test = y_train[~y_train.index.isin(testList)], y_train[y_train.index.isin(testList)]

  return X_train, X_test, y_train, y_test

In [16]:
def createClassifier(modelName, parameters):
  if modelName == "KNN":
    return KNeighborsClassifier(parameters) if parameters is not None else KNeighborsClassifier()
  elif modelName == "LR":
    return LogisticRegression(parameters) if parameters is not None else LogisticRegression()
  elif modelName == "GNB":
    return GaussianNB(parameters) if parameters is not None else GaussianNB()
  elif modelName == "DT":
    return DecisionTreeClassifier(parameters) if parameters is not None else DecisionTreeClassifier()
  elif modelName == "SVM":
    return SVC(parameters) if parameters is not None else SVC()
  elif modelName == "RF":
    return RandomForestClassifier(parameters) if parameters is not None else RandomForestClassifier()
  elif modelName == "GB":
    return GradientBoostingClassifier(parameters) if parameters is not None else GradientBoostingClassifier()
  else:
    return KNeighborsClassifier()

In [17]:
def trainModel(X_train, X_test, y_train, y_test, modelName, parameters = None):
  # Creación del clasificador KNN
  clf = createClassifier(modelName, parameters)

  # Entrenamiento del clasificador KNN
  clf.fit(X_train, y_train)

  # Calculando la predicción del modelo con la data de prueba
  y_pred = clf.predict(X_test)

  return y_test, y_pred

In [18]:
def getMetrics(y_train, y_test, y_pred, startDate, endDate, partitionNumber, algorithm = "KNN"):
  trainRows = len(y_train)
  testRows = len(y_test)

  # Calculando la exactitud del modelo
  accuracy = "{:.2%}".format(accuracy_score(y_test, y_pred))

  # Calculando la precisión del modelo
  precision = "{:.2%}".format(precision_score(y_test, y_pred))

  # Calculando la sensibilidad del modelo
  recall = "{:.2%}".format(recall_score(y_test, y_pred))

  # Calculando el valor F del modelo (robustez)
  f1Score = "{:.2%}".format(f1_score(y_test, y_pred))

  # Calculando el tiempo de ejecución del modelo
  executionTime = str(int((endDate - startDate).total_seconds() * 1000)) + "ms"

  confussionMatrix = str(confusion_matrix(y_test, y_pred).tolist())
  
  return {
    "algoritmo": algorithm,
    "tipo": "Total de datos" if partitionNumber == 0 else "Particion " + str(partitionNumber),
    "registrosEntrenamiento": trainRows,
    "registrosPrueba": testRows,
    "tiempoEjecucion": executionTime,
    "matrizConfusion": confussionMatrix,
    "exactitud": accuracy,
    "precision": precision,
    "sensibilidad": recall,
    "valorF": f1Score,
  }


In [19]:
def main():
  # Definiendo el inicio del proceso
  startTime = datetime.datetime.now()
  print("Inicio: " + str(startTime))
  print("Se inició el procesamiento")
  print()

  # Leyendo la data
  data = readData(os.path.join(inputFolder, "result.csv"), ',', 'utf-8', 0)

  # Determinando los dataframes de las categorías (X) y el objetivo (y)
  X, y = preprocessingData(data)

  # Creando el arreglo de metricas de cada partición
  metricsList = []

  # Iterando sobre cada partición
  for partitionNumber in range(0,partitionsNumber+1):
    # Separando data para el entrenamiento y testeo
    X_train, X_test, y_train, y_test = splitData(X, y, partitionNumber)

    models = ["KNN", "LR", "GNB", "DT", "SVM", "RF", "GB"]

    for model in models:
      print("Calculando para el algoritmo {}".format(model))

      # Inicio de ejecución del modelo
      startDate = datetime.datetime.now()
      print("Inicio: " + str(startDate))
      
      # Realizar entrenamiento del modelo
      y_test, y_pred = trainModel(X_train, X_test, y_train, y_test, model)

      # Fin de ejecución del modelo
      endDate = datetime.datetime.now()
      print("Fin: " + str(endDate))
      print("Tiempo: " + str(endDate-startDate))
      print()

      # Obteniendo las métricas de la partición del modelo
      metrics = getMetrics(y_train, y_test, y_pred, startDate, endDate, partitionNumber, model)

      # Añadiendo la métrica de la partición a la lista de métricas
      metricsList.append(metrics)

  # Ordenando las métricas
  metricsList = sorted(metricsList, key=lambda x: (x["algoritmo"], x["tipo"]))
    
  # Escribiendo las metricas en un archivo de salida
  outputFileDateTime = datetime.datetime.now()
  writeJson(metricsList, os.path.join(outputFolder, "json", outputFileDateTime.strftime("%Y-%m-%d %H-%M-%S") + ".json"))
  writeCsv(metricsList, os.path.join(outputFolder, "csv", outputFileDateTime.strftime("%Y-%m-%d %H-%M-%S") + ".csv"))

  # Definiendo el fin del proceso
  endTime = datetime.datetime.now()
  print("Fin: " + str(endTime))
  print("Tiempo: " + str(endTime-startTime))
  
  # Retornando la lista de métricas
  return metricsList

In [20]:
if __name__ == "__main__":
  main()

Inicio: 2023-05-23 22:48:40.398962
Se inició el procesamiento

Calculando para el algoritmo KNN
Inicio: 2023-05-23 22:48:45.023138
Fin: 2023-05-23 22:48:45.501982
Tiempo: 0:00:00.478844

Calculando para el algoritmo LR
Inicio: 2023-05-23 22:48:45.513942


c:\Users\Ronaldo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fin: 2023-05-23 22:48:46.058160
Tiempo: 0:00:00.544218

Calculando para el algoritmo GNB
Inicio: 2023-05-23 22:48:46.068125
Fin: 2023-05-23 22:48:46.227670
Tiempo: 0:00:00.159545

Calculando para el algoritmo DT
Inicio: 2023-05-23 22:48:46.236175
Fin: 2023-05-23 22:48:46.837939
Tiempo: 0:00:00.601764

Calculando para el algoritmo SVM
Inicio: 2023-05-23 22:48:46.847441
Fin: 2023-05-23 22:49:33.242055
Tiempo: 0:00:46.394614

Calculando para el algoritmo RF
Inicio: 2023-05-23 22:49:33.250890
Fin: 2023-05-23 22:49:36.749151
Tiempo: 0:00:03.498261

Calculando para el algoritmo GB
Inicio: 2023-05-23 22:49:36.758123
Fin: 2023-05-23 22:49:44.377045
Tiempo: 0:00:07.618922

Calculando para el algoritmo KNN
Inicio: 2023-05-23 22:49:44.478075
Fin: 2023-05-23 22:49:44.789360
Tiempo: 0:00:00.311285

Calculando para el algoritmo LR
Inicio: 2023-05-23 22:49:44.798329


c:\Users\Ronaldo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fin: 2023-05-23 22:49:45.178367
Tiempo: 0:00:00.380038

Calculando para el algoritmo GNB
Inicio: 2023-05-23 22:49:45.187336
Fin: 2023-05-23 22:49:45.314743
Tiempo: 0:00:00.127407

Calculando para el algoritmo DT
Inicio: 2023-05-23 22:49:45.323713
Fin: 2023-05-23 22:49:45.690166
Tiempo: 0:00:00.366453

Calculando para el algoritmo SVM
Inicio: 2023-05-23 22:49:45.698139
Fin: 2023-05-23 22:50:11.776124
Tiempo: 0:00:26.077985

Calculando para el algoritmo RF
Inicio: 2023-05-23 22:50:11.784085
Fin: 2023-05-23 22:50:14.404362
Tiempo: 0:00:02.620277

Calculando para el algoritmo GB
Inicio: 2023-05-23 22:50:14.412233
Fin: 2023-05-23 22:50:20.252148
Tiempo: 0:00:05.839915

Calculando para el algoritmo KNN
Inicio: 2023-05-23 22:50:20.351612
Fin: 2023-05-23 22:50:20.630342
Tiempo: 0:00:00.278730

Calculando para el algoritmo LR
Inicio: 2023-05-23 22:50:20.639312


...

Fin: 2023-05-23 22:50:21.019053
Tiempo: 0:00:00.379741

Calculando para el algoritmo GNB
Inicio: 2023-05-23 22:50:21.028022
Fin: 2023-05-23 22:50:21.143118
Tiempo: 0:00:00.115096

Calculando para el algoritmo DT
Inicio: 2023-05-23 22:50:21.150095
Fin: 2023-05-23 22:50:21.540168
Tiempo: 0:00:00.390073

Calculando para el algoritmo SVM
Inicio: 2023-05-23 22:50:21.548141
Fin: 2023-05-23 22:50:47.449578
Tiempo: 0:00:25.901437

Calculando para el algoritmo RF
Inicio: 2023-05-23 22:50:47.457441
Fin: 2023-05-23 22:50:50.168718
Tiempo: 0:00:02.711277

Calculando para el algoritmo GB
Inicio: 2023-05-23 22:50:50.176222
Fin: 2023-05-23 22:50:56.055944
Tiempo: 0:00:05.879722

Calculando para el algoritmo KNN
Inicio: 2023-05-23 22:50:56.154161
Fin: 2023-05-23 22:50:56.431241
Tiempo: 0:00:00.277080

Calculando para el algoritmo LR
Inicio: 2023-05-23 22:50:56.441208


c:\Users\Ronaldo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fin: 2023-05-23 22:50:56.826612
Tiempo: 0:00:00.385404

Calculando para el algoritmo GNB
Inicio: 2023-05-23 22:50:56.834584
Fin: 2023-05-23 22:50:56.947904
Tiempo: 0:00:00.113320

Calculando para el algoritmo DT
Inicio: 2023-05-23 22:50:56.954881
Fin: 2023-05-23 22:50:57.361760
Tiempo: 0:00:00.406879

Calculando para el algoritmo SVM
Inicio: 2023-05-23 22:50:57.369735
Fin: 2023-05-23 22:51:23.991427
Tiempo: 0:00:26.621692

Calculando para el algoritmo RF
Inicio: 2023-05-23 22:51:23.999635
Fin: 2023-05-23 22:51:26.811575
Tiempo: 0:00:02.811940

Calculando para el algoritmo GB
Inicio: 2023-05-23 22:51:26.819547
Fin: 2023-05-23 22:51:32.733485
Tiempo: 0:00:05.913938

Calculando para el algoritmo KNN
Inicio: 2023-05-23 22:51:32.830385
Fin: 2023-05-23 22:51:33.110061
Tiempo: 0:00:00.279676

Calculando para el algoritmo LR
Inicio: 2023-05-23 22:51:33.120027


c:\Users\Ronaldo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fin: 2023-05-23 22:51:33.527591
Tiempo: 0:00:00.407564

Calculando para el algoritmo GNB
Inicio: 2023-05-23 22:51:33.536560
Fin: 2023-05-23 22:51:33.648283
Tiempo: 0:00:00.111723

Calculando para el algoritmo DT
Inicio: 2023-05-23 22:51:33.656259
Fin: 2023-05-23 22:51:33.983258
Tiempo: 0:00:00.326999

Calculando para el algoritmo SVM
Inicio: 2023-05-23 22:51:33.991233
Fin: 2023-05-23 22:52:00.821164
Tiempo: 0:00:26.829931

Calculando para el algoritmo RF
Inicio: 2023-05-23 22:52:00.829353
Fin: 2023-05-23 22:52:03.510408
Tiempo: 0:00:02.681055

Calculando para el algoritmo GB
Inicio: 2023-05-23 22:52:03.517384
Fin: 2023-05-23 22:52:09.385307
Tiempo: 0:00:05.867923

Fin: 2023-05-23 22:52:09.394651
Tiempo: 0:03:28.995689
